# 2022年5月17日 Julia学习笔记
## 数据类型转换
数据类型转换应该显式实现。但由于实现了一组默认规则，使得他们可以自动进行转换。
### 执行简单的数据类型转换
常见数据类型转换有如下两种：

In [4]:
Float64(1//3) 
# 这里两个斜杠和一个斜杠有什么区别？

0.3333333333333333

或者采用convert函数来进行转换。

In [5]:
convert(Float64, 1//3)

0.3333333333333333

上述两种方法都就可以正常工作。考虑性能优化时，convert会更有优势。
### 2.7.2 注意有损转换
数据类型转换并不是无损的，所以没事别瞎转。

In [6]:
convert(Rational, convert(Float64, 1//3))

6004799503160661//18014398509481984

只要用Float64，就肯定会有这个问题。如果需要更高精度，可以改用BigFloat。

**在处理浮点数变量时，应该谨慎对待精度问题。**

### 理解数字类型转换

出于安全原因，Julia不会自动对数据类型执行转换。每次转换都必须由开发人员显式地指定。
为了便于实现，Julia默认情况下已经包含数字类型的转换函数，例如，可以从Base包中发现以下代码：
```Julia
convert(::Type{T}, x::T) where {T<:Number} = x
convert(::Type{T}, x::Number) where {T<:Number} = T(x)
```
这两个函数的第一个参数都是Type{T}。

第一个函数表示，只要T时Number的子类型，当我们想将x从T类型转换为T类型时(相同类型转换，这不是撑的么？不是，有可能多个类型批量转成一个类型~)，很容易返回参数x本身，这可以视为性能优化，因为当目标类型与输入相同时，实际上不需要进行任何转换。

第二个函数表示为了将Number的子类型变量x转换为同为Number子类型T的类型，仅使用x调用T类型的构造函数。

### 重温自动转换规则
在以下情况下会自动调用convert函数：
1. 赋值给数组会将值转换为该数组的元素类型。

In [7]:
x = rand(3)
x[1] = 1
x

3-element Vector{Float64}:
 1.0
 0.8089462884033066
 0.04907157113968197

2. 赋值给对象的字段会将值转换为该字段的声明类型。

In [3]:
mutable struct Foo
    x::Float64
end
foo = Foo(1)
foo.x = 2
foo

Foo(1.0)

3. 使用new构造对象会将值转换为该对象的声明的字段类型。

In [1]:
struct Foo
    x::Float64
    Foo(v) = new(v)
end
Foo(1)

Foo(1.0)

4. 赋值给具有声明类型的变量会将值转换为该类型。

In [1]:
function foo()
    local x::Float64
    x = 1
    println(x, " has type of ", typeof(x))
end

foo (generic function with 1 method)

In [2]:
foo()

1.0 has type of Float64


5. 具有声明的返回类型的函数会将其返回值转换为该类型。

In [3]:
function foo()::Float64
    return 1
end

foo (generic function with 1 method)

In [4]:
foo()

1.0

6. 将值传递给ccall会将值转换为相应的参数类型。

In [ ]:
ccall((:exp), Float64, (Float64,), 2)

上面语句中，原文为：
```Julia
ccall((:exp, "libc"), Float64, (Float64,), 2)
```
然而运行没有通过。

### 理解函数分派规则
仅当函数的参数类型正确匹配时，才能调用该函数。可以将适当的匹配定义为完全匹配的匹配，或者将要传递的参数是函数签名中期望的子类型时匹配。

为了说明这一点，创建一个函数，将其AbstractFloat类型的参数值加倍。

In [8]:
function subtypetree(roottype,level = 1, indent = 4)
    level == 1 && println(roottype)
    for s in subtypes(roottype)
        println(join(fill(" ", level * indent)) * string(s))
        subtypetree(s, level + 1, indent)
    end
end

subtypetree (generic function with 3 methods)

In [9]:
subtypetree(AbstractFloat)

AbstractFloat
    BigFloat
    Float16
    Float32
    Float64


In [10]:
twice(x::AbstractFloat) = 2x

twice (generic function with 1 method)

In [11]:
twice(1.0)

2.0

In [12]:
twice(2)

MethodError: MethodError: no method matching twice(::Int64)
Closest candidates are:
  twice(!Matched::AbstractFloat) at d:\Git\learning-julia\20220517_数据类型转换.ipynb:1

如果希望函数能够自动适应参数类型，那么更通用地编写函数即可解决此问题。

In [13]:
twice(x) = 2x

twice (generic function with 2 methods)

In [14]:
twice(2)

4

In [16]:
twice(1.0)

2.0

如果参数必须是Number，则可以限制为：

In [18]:
twice(x::Number) = 2x

twice (generic function with 3 methods)

In [19]:
twice(2.0)

4.0

In [20]:
twice(2//3)

4//3